In [80]:
import numpy as np
import pywt
import os
import pandas as pd

In [57]:


# Specify the paths to the directories containing the xlsx files
directory2 = r"C:\Users\User\Desktop\supplement materials_portfolios\400_raw_stock_data_part2"
directory1 = r"C:\Users\User\Desktop\supplement materials_portfolios\400_stocks\400_raw_stock_data_part1"

directory_test = r"C:\Users\User\Desktop\supplement materials_portfolios\test_folder"
# Initialize an empty list to store the dataframes
dataframes = []

# Read the xlsx files from directory1
for filename in os.listdir(directory_test):
    if filename.endswith('.xlsx'):
        filepath = os.path.join(directory_test, filename)
        df = pd.read_excel(filepath)
        dataframes.append(df)

"""# Read the xlsx files from directory2
for filename in os.listdir(directory2):
    if filename.endswith('.xlsx'):
        filepath = os.path.join(directory2, filename)
        df = pd.read_excel(filepath)
        dataframes.append(df)"""

# Concatenate the dataframes into a single dataframe
combined_df = pd.concat(dataframes)

# Now you can work with the combined dataframe


In [58]:
combined_df

,Unnamed: 0,ts_code,trade_date,open,high,low,close,pre_close,change,pct_chg,vol,amount
0,0,000001.SZ,20201203,2196.5294,2205.4133,2128.7902,2169.8779,2179.8722,-9.9943,-0.4585,714452.30,1.393085e+06
1,1,000001.SZ,20201202,2213.1866,2227.6229,2167.6570,2179.8722,2226.5124,-46.6402,-2.0948,889385.29,1.758639e+06
2,2,000001.SZ,20201201,2187.6456,2277.5945,2154.3312,2226.5124,2192.0875,34.4249,1.5704,1263719.75,2.516011e+06
3,3,000001.SZ,20201130,2209.8552,2318.6822,2175.4303,2192.0875,2187.6456,4.4419,0.2030,1581441.28,3.213680e+06
4,4,000001.SZ,20201127,2220.9600,2220.9600,2152.1102,2187.6456,2165.4360,22.2096,1.0256,753773.74,1.479431e+06
...,...,...,...,...,...,...,...,...,...,...,...,...
4590,4590,000524.SZ,20010108,32.7360,32.8350,31.8120,32.6370,32.5380,0.0990,0.3043,16588.59,1.624770e+04
4591,4591,000524.SZ,20010105,31.8450,32.6700,31.8450,32.5380,31.8450,0.6930,2.1762,18557.51,1.814258e+04
4592,4592,000524.SZ,20010104,32.2410,32.2740,31.7460,31.8450,32.1750,-0.3300,-1.0256,9812.00,9.510954e+03
4593,4593,000524.SZ,20010103,32.1750,32.3400,31.7790,32.1750,32.0430,0.1320,0.4119,13835.57,1.346716e+04


In [109]:
ts_c = '000524.SZ'
stock_data = combined_df[combined_df['ts_code'] == ts_c].sort_values(by='trade_date')

In [106]:
trimmed_stock_data = stock_data.iloc[:3000]

In [112]:
MS = pd.DataFrame()
for index, row in trimmed_stock_data.iterrows():
    MS[row['trade_date']] = [row['close']]
MS.index = [ts_c]
MS

C:\Users\User\AppData\Local\Temp\ipykernel_10364\2652660893.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  MS[row['trade_date']] = [row['close']]


,20010102,20010103,20010104,20010105,20010108,20010109,20010110,20010111,20010112,20010115,...,20131120,20131121,20131122,20131125,20131126,20131127,20131128,20131129,20131202,20131203
000524.SZ,32.043,32.175,31.845,32.538,32.637,32.373,33.099,32.571,32.01,30.954,...,27.3502,27.0358,26.7664,26.946,26.6765,27.7993,27.4849,27.4849,25.5987,26.2274


In [95]:

# Define the denoising function
def dwt_denoise(data):
    coeffs = pywt.wavedec(data, 'db4', level=5)
    threshold = 0.1 * max(coeffs[0])
    
    # Apply soft thresholding to the detail coefficients
    denoised_coeffs = [pywt.threshold(c, threshold, mode='soft') for c in coeffs]
    
    # Reconstruct the denoised data
    denoised_data = pywt.waverec(denoised_coeffs, 'db4')
    
    return denoised_data


test_close2 = dwt_denoise(test_close)
print(test_close2)

C:\Users\User\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\pywt\_multilevel.py:43: UserWarning: Level value of 5 is too high: all coefficients will experience boundary effects.
  warnings.warn(


ValueError: setting an array element with a sequence.

In [ ]:
def compute_revised_return(X, Y):
    RT = []
    RC = []
    
    for i in range(len(X)):
        temp_vec_rt = []
        temp_vec_rc = []
        longhold = 0
        
        for j in range(len(Y[i])):
            if j == 0:
                return_rt = 0
            else:
                return_rt = (X[i][j] - X[i][j-1]) / X[i][j-1]
            
            temp_vec_rt.append(return_rt)
            
            if longhold > 0:
                return_rc = (X[i][j] - X[i][j-1]) / X[i][j-1]
            
            if longhold == 0:
                return_rc = 0
            
            if longhold == 0 and Y[i][j] > 0:
                longhold = 1
                return_rc = 0
            
            if longhold > 0 and (Y[i][j] > 0 or j == len(Y[i])-1):
                longhold = 0
                return_rc = (X[i][j] - X[i][j-1]) / X[i][j-1]
            
            temp_vec_rc.append(return_rc)
        
        RT.append(temp_vec_rt)
        RC.append(temp_vec_rc)
    
    return RT, RC
